In [ ]:
# import all needed libraries
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
import random

In [ ]:
# Create the geometry
domain = Rectangle(12,3).Face()
domain.edges.Min(X).name = "left"
domain.edges.Max(X).name = "right"
domain.edges.Min(Y).name = "bottom"
domain.edges.Max(Y).name = "top"
geo = OCCGeometry(domain, dim=2)

In [ ]:
# Generate a mesh
mesh = Mesh(geo.GenerateMesh(maxh=0.08))
Draw(mesh)

In [ ]:
# Create finite element space on the mesh
order = 2
fes = H1(mesh, order=order) * H1(mesh, order=order)
(p,T),(q,v) = fes.TnT()

In [ ]:
# Define constants
anisotropy = True
if anisotropy:
    epsbar = 0.01
    j = 4
    delta = 0.05
    theta0 = 0
else:
    eps = 0.01
tau = 0.003
gamma = 10
alpha = 0.9
K = 1.2
a1_val = 0.01
random.seed(1)
random_val = random.random()

timestep = 0.001
print("random = ", random_val)

In [ ]:
# Create solution objects
gf, gfold = GridFunction(fes), GridFunction(fes)

gfp, gfT = gf.components
gfpold, gfTold = gfold.components

Consider the following equations for $p$ and $T$:

$$
\tau \partial_t p = \varepsilon^2 \Delta p + p(1-p)(p-0.5 + \frac{\alpha}{\pi} \text{atan}(\gamma (1-T)) + a_1 p (1-p)(0.5 - r) = 0
$$

$$
\partial_t T = \Delta T + K \partial_t p
$$

Using $\partial_t p \approx dt^{-1}(p-p_o)$, integrating over the domain, partial integration on $\Delta$ parts and bringing everything to the left hand side we get

$$
\int_\Omega (p-p_o) + dt \varepsilon^2 \nabla p \nabla q - dt p_o(1-p_o)(p_o-0.5+\frac{\alpha}{\pi} atan(\gamma (1-T_o)) q - dt a_1 p_o (1-p_o) (0.5 - r) \, dx = 0
$$

$$
\int_\Omega (T-T_o) + dt \nabla T \nabla v + K (p-p_o) v \, dx = 0
$$

We disretize this in time with an explicit euler scheme:

In [ ]:
theta = theta = IfPos(InnerProduct(grad(gfpold),grad(gfpold)),acos(grad(gfpold)[1]/sqrt(InnerProduct(grad(gfpold),grad(gfpold)))),0)
eps = epsbar*(1+delta*cos(j*(theta-theta0)))

a = BilinearForm(fes)
a += tau * (p-gfpold) * q * dx
a += timestep * eps**2 * grad(p) * grad(q) *dx
a += - timestep * gfpold*(1-gfpold)*(gfpold-0.5+alpha/pi * atan(gamma * (1-gfTold))) * q * dx
a += - timestep * a1_val * gfpold * (1-gfpold) * (0.5 - random_val) * q * dx
a += (T-gfTold) * v * dx 
a += timestep * grad(T)*grad(v) * dx
a += - K * (p-gfpold)*v * dx

In [ ]:
b = BilinearForm(fes)
b += - K * p * v * dx

As one can see, the first equation does not contain a $T$ and determines $p$, therefore we can compute first the new solution $p$ using the first equation block, then update the residual for the second block and solve for $T$:

In [ ]:
# Compute inverse of subblocks
with TaskManager():
    a.AssembleLinearization(gf.vec)
    bits0 = BitArray(fes.FreeDofs())
    bits0[fes.Range(1)] = False
    bits1 = BitArray(fes.FreeDofs())
    bits1[fes.Range(0)] = False
    inv1 = a.mat.Inverse(bits0)
    inv2 = a.mat.Inverse(bits1)
    b.Assemble()
    res = gf.vec.CreateVector()
    tmp = gf.vec.CreateVector()
    tmp1 = tmp.CreateVector()

In [ ]:
# Timestepping
t = 0
tend = 30

print("Phase:")
scenep = Draw(gfp, mesh, autoscale=False, min=0, max=1)
print("Temperature")
sceneT = Draw(gfT, mesh, autoscale=False, min=0, max=1)

count = 0

# Set initial condition
gfT.vec[:] = 0.
gfp.Set(1, definedon=mesh.Boundaries('left'))

with TaskManager():
    print(t, tend * (1-1e-8))
    while t < tend * (1-1e-8):
        t += timestep
        print(f"t = {t:.3}", end="\r")
        count += 1
        gfold.vec.data = gf.vec
        a.Apply(gf.vec, res)
        tmp1.data = inv1 * res
        gf.vec.data -= tmp1
        tmp.data = b.mat * tmp1
        res.data -= tmp
        gf.vec.data -= inv2 * res
        # Redraw is expensive so do it only every 10th timestep
        if count % 10 == 0:
            scenep.Redraw()
            sceneT.Redraw()

scenep.Redraw()
sceneT.Redraw()